## Carga y Transformación

In [1]:
# Importamos librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler,  OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from joblib import dump

In [2]:
# Cargamos los datos
Data = pd.read_csv("../Datos/data_adults.csv")
Data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# Vista rápida de los datos
print(Data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None


## 2. Preprocesamientos

In [4]:
# Eliminamos columnas irrelevantes
Data_cop = Data.drop("fnlwgt", axis=1)
Data_cop = Data_cop.drop("education-num", axis=1)

In [5]:
# Separación de Variables (X) y Objetivo (Y)
X = Data_cop.drop("income", axis=1)
y = Data_cop['income'].isin(['>50K.','>50K'])

In [6]:
# Definimos las variables categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

# Creamos pipelines de preprocesamiento
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Usamos ColumnTransformer para combinar ambas transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [7]:
# Definimos el pipeline completo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

## Entrenamiento Modelo con Pipeline

In [8]:
# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un nuevo pipeline que incluye el preprocesamiento y el modelo
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))])

# Entrenamos el pipeline completo en los datos de entrenamiento
model_pipeline.fit(X_train, y_train)

# Hacemos predicciones en el conjunto de prueba
y_pred = model_pipeline.predict(X_test)

# Evaluamos el rendimiento del modelo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.88      0.92      0.90      7414
        True       0.71      0.61      0.66      2355

    accuracy                           0.85      9769
   macro avg       0.80      0.77      0.78      9769
weighted avg       0.84      0.85      0.84      9769



## 3. Optimización de Hiperparametros de RandomForest

In [9]:
param_grid={
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.01, 0.1, 0.2, 0.5, 0.7],
    'max_depth': [3, 5, 7, 10, 15],
    'subsample': [0.8, 1.0]
}
model_gb=GradientBoostingClassifier(random_state=2024)

#### Instanciamos la busqueda del mejor modelo

model_busqueda=GridSearchCV(estimator=model_gb,
                            param_grid=param_grid,
                            cv=3,
                            verbose=4,
                            scoring='f1',
                            n_jobs=-1)

model_pipeline_train= Pipeline(steps=[('preprocessor', preprocessor),
                                 ('busqueda', model_busqueda)])


In [10]:
# Entrenamos el pipeline completo en los datos de entrenamiento
model_pipeline_train.fit(X_train, y_train)

Fitting 3 folds for each of 250 candidates, totalling 750 fits


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'capital-gain', 'capital-loss', 'hours-per-week'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object'))])),
                ('busqueda',
                 GridSearchCV(cv=3,
                              estimator=GradientBoostingClassifier(random_state=2024),
                              n_jobs=-1,
                              param_grid={'learning_rate': [0.01, 0.1, 0.2, 0.5,
                                                            0.7],
                                          'max_depth': [3, 5, 7, 10, 15],
                                          'n_estimators': [50, 100, 150, 200,
                                                           250],
                                          'subsample': [0.8, 1.0]},
                              scoring='f1', verbose=4))])

## Busqueda de los mejores Hiperparametros

In [11]:
busqueda_resultado=model_pipeline_train['busqueda']
busqueda_resultado.best_estimator_.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.2,
 'loss': 'log_loss',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 150,
 'n_iter_no_change': None,
 'random_state': 2024,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [12]:
# Score del modelo con los mejores Hiperparametros con los datos test
y_pred=model_pipeline_train.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.89      0.95      0.92      7414
        True       0.79      0.64      0.71      2355

    accuracy                           0.87      9769
   macro avg       0.84      0.79      0.81      9769
weighted avg       0.87      0.87      0.87      9769



In [13]:
# Score del primero modelo entrenado con los datos de test
y_pred=model_pipeline.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.88      0.92      0.90      7414
        True       0.71      0.61      0.66      2355

    accuracy                           0.85      9769
   macro avg       0.80      0.77      0.78      9769
weighted avg       0.84      0.85      0.84      9769



In [14]:
# Score del modelo con los mejores Hiperparametros con los datos entrenamiento
y_pred_train=model_pipeline_train.predict(X_train)
print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

       False       0.91      0.95      0.93     29741
        True       0.82      0.70      0.75      9332

    accuracy                           0.89     39073
   macro avg       0.87      0.82      0.84     39073
weighted avg       0.89      0.89      0.89     39073



In [15]:
# Score del primero modelo entrenado con los datos de entrenamiento
y_pred_train=model_pipeline.predict(X_train)
print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

       False       0.98      0.99      0.98     29741
        True       0.96      0.93      0.94      9332

    accuracy                           0.97     39073
   macro avg       0.97      0.96      0.96     39073
weighted avg       0.97      0.97      0.97     39073



In [16]:
# Hiperparametros del primer modelo entrenado
model_pipeline['classifier'].get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}